In [1]:
pip install transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import string
import re
import scipy.sparse as sp

import nltk #using the nltk library for tokenization and lemmatization
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from transformers import BertTokenizer, BertModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import vstack, hstack
from sklearn.metrics.pairwise import cosine_similarity

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_3310/4035032612.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords

In [4]:
bx_cleaned = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/bx_cleaned')

In [5]:
gbx_cleaned = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/gbx_cleaned')

In [6]:
#Function to clean text data
def clean_text(text):
    text = text.lower()     #Converting the text to lowercase
    text = re.sub(r'\d+', ' ', text)    # Removing digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Removing special characters
    text = text.translate(str.maketrans('', '', string.punctuation))    #Removing punctuation     
    tokens = word_tokenize(text) #Tokenization of the text
    non_stopwords = []  #Creating an empty list to store the non-stopwords
    stop_words = set(stopwords.words('english')) #Initializing the NLTK English stopwords
    #Lemmatization of the words/tokens
    lemmatizer = WordNetLemmatizer()    #Initialization of the NLTK WordNet Lemmatizer

    #Iterating through the list of tokens, lemmatizing them, and adding it to the list of non-stopwords
    for token in tokens:
        if token not in stop_words:
            lemma = lemmatizer.lemmatize(token) #Performing lemmatization
            non_stopwords.append(lemma)  # Appending the stemmed token back to the list of non-stopwords

    # Joining the tokens/words from the list of non-stopwords into a string
    filtered_text = ' '.join(non_stopwords)
    return filtered_text

In [7]:
def clean_genres(text):
    text = text.lower()
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [8]:
bx_cleaned['cleaned_genres'] = bx_cleaned['Genres'].apply(clean_genres)

In [9]:
gbx_cleaned['cleaned_genres'] = gbx_cleaned['Genres'].apply(clean_genres)

In [10]:
bx_cleaned['title_genres_combined'] = bx_cleaned['cleaned_title'] + ' ' + bx_cleaned['cleaned_genres']
gbx_cleaned['title_genres_combined'] = gbx_cleaned['cleaned_title'] + ' ' + gbx_cleaned['cleaned_genres']

In [11]:
# Initializing BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [12]:
# Generate BERT embeddings for a list of texts in batches

def get_bert_embeddings_batch(text_list, batch_size=32):
    embeddings = [] #initializign an empty list to store the BERT embeddings for all the texts
    for i in range(0, len(text_list), batch_size):  #batch processing the texts from the list of texts
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512) #tokenization of the texts in the batch and returning the PyTorch tensors
        outputs = model(**inputs)   #passing the tokenized inputs to the BERT model to retrieve the outputs
        embeddings.extend(outputs.last_hidden_state.mean(dim=1).detach().numpy()) 
    return np.vstack(embeddings)    #vertical stacking of of the embeddings to create a single matrix of embeddings for the input texts

In [13]:
# unique_author_bx = bx_cleaned['Author'].drop_duplicates().reset_index(drop=True)
# author_embeddings_bx = get_bert_embeddings_batch(bx_cleaned['Author'].tolist())
# genres_to_embedding_bx = dict(zip(unique_author_bx, author_embeddings_bx))
# # bx_cleaned.head(3)

In [14]:
# unique_author_gbx = gbx_cleaned['Author'].drop_duplicates().reset_index(drop=True)
# author_embeddings_gbx = get_bert_embeddings_batch(gbx_cleaned['Author'].tolist())
# genres_to_embedding_gbx = dict(zip(unique_author_gbx, author_embeddings_gbx))
# # gbx_cleaned.head(3)

In [15]:
# unique_author_bx.to_csv('unique_author_bx.csv', index=False)
# unique_author_gbx.to_csv('unique_author_gbx.csv', index=False)

In [16]:
# np.save('author_embeddings_bx.npy', author_embeddings_bx)
# np.save('author_embeddings_gbx.npy', author_embeddings_gbx)

### Saving the data frames as csv files

In [17]:
# gbx_cleaned.to_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/gbx_cleaned', index=False)

In [18]:
# bx_cleaned.to_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/bx_cleaned', index=False)

### Loading the embeddings from the Numpys

In [19]:
bx_title_embeddings_array = np.load('title_embeddings_bx.npy')
gbx_title_embeddings_array = np.load('title_embeddings_gbx.npy')
bx_combined_embeddings_array = np.load('combined_embeddings_bx.npy')
gbx_combined_embeddings_array = np.load('combined_embeddings_gbx.npy')
bx_genres_embeddings_array = np.load('genres_embeddings_bx.npy')
gbx_genres_embeddings_array = np.load('genres_embeddings_gbx.npy')

bx_authors_embeddings_array = np.load('author_embeddings_bx.npy')
gbx_authors_embeddings_array = np.load('author_embeddings_gbx.npy')

### Loading the unique embeddings

In [20]:
unique_titles_gbx = pd.read_csv('unique_titles_gbx.csv')['Title']
unique_titles_bx = pd.read_csv('unique_titles_bx.csv')['Title']
unique_combined_gbx = pd.read_csv('unique_combined_gbx.csv')['new_combined']
unique_combined_bx = pd.read_csv('unique_combined_bx.csv')['new_combined']
unique_genres_gbx = pd.read_csv('unique_genres_gbx.csv')['Genres']
unique_genres_bx = pd.read_csv('unique_genres_bx.csv')['Genres']

unique_authors_bx = pd.read_csv('unique_author_bx.csv')['Author']
unique_authors_gbx = pd.read_csv('unique_author_gbx.csv')['Author']

### Updating the data frame with the embedding values

In [21]:
title_to_embedding_gbx = dict(zip(unique_titles_gbx, gbx_title_embeddings_array))
gbx_cleaned['title_embeddings'] = gbx_cleaned['Title'].map(title_to_embedding_gbx)

title_to_embedding_bx = dict(zip(unique_titles_bx, bx_title_embeddings_array))
bx_cleaned['title_embeddings'] = bx_cleaned['Title'].map(title_to_embedding_bx)

combined_to_embedding_gbx = dict(zip(unique_combined_gbx, gbx_combined_embeddings_array))
gbx_cleaned['combined_embeddings'] = gbx_cleaned['new_combined'].map(combined_to_embedding_gbx)

combined_to_embedding_bx = dict(zip(unique_combined_bx, bx_combined_embeddings_array))
bx_cleaned['combined_embeddings'] = bx_cleaned['new_combined'].map(combined_to_embedding_bx)

genres_to_embedding_gbx = dict(zip(unique_genres_gbx, gbx_genres_embeddings_array))
gbx_cleaned['genres_embeddings'] = gbx_cleaned['Genres'].map(genres_to_embedding_gbx)

genres_to_embedding_bx = dict(zip(unique_genres_bx, bx_genres_embeddings_array))
bx_cleaned['genres_embeddings'] = bx_cleaned['Genres'].map(genres_to_embedding_bx)

authors_to_embedding_bx = dict(zip(unique_authors_bx, bx_authors_embeddings_array))
bx_cleaned['authors_embeddings'] = bx_cleaned['Author'].map(authors_to_embedding_bx)

authors_to_embedding_gbx = dict(zip(unique_authors_gbx, gbx_authors_embeddings_array))
gbx_cleaned['authors_embeddings'] = gbx_cleaned['Author'].map(authors_to_embedding_gbx)

# Content-based Model

##### Using BERT embeddings, TFIDF vectorization, and kNN

In [22]:
relevant_columns = ['Title', 'Author', 'Genres', 'cleaned_title', 'cleaned_author', 'new_combined', 'title_embeddings', 'authors_embeddings', 'genres_embeddings', 'combined_embeddings', 'cleaned_genres']  

In [23]:
relevant_columns_gbx = ['BookID', 'Title', 'Author', 'Genres', 'cleaned_title', 'cleaned_author', 'new_combined', 'title_embeddings', 'authors_embeddings', 'genres_embeddings', 'combined_embeddings', 'cleaned_genres']  

In [24]:
# Drop duplicates based on the 'Title' column and select only the relevant columns for bx_cleaned and gbx_cleaned
gbx_content = gbx_cleaned.drop_duplicates(subset='Title')[relevant_columns_gbx]
bx_content = bx_cleaned.drop_duplicates(subset='Title')[relevant_columns]

In [25]:
gbx_content.head(1)

,BookID,Title,Author,Genres,cleaned_title,cleaned_author,new_combined,title_embeddings,authors_embeddings,genres_embeddings,combined_embeddings,cleaned_genres
0,2767052,The Hunger Games,Suzanne Collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game,suzanne collins,hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.12804408, 0.11276019, -0.25651056, -0.0526...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...",young adult fiction fantasy dystopia science f...


In [26]:
bx_content.head(1)

,Title,Author,Genres,cleaned_title,cleaned_author,new_combined,title_embeddings,authors_embeddings,genres_embeddings,combined_embeddings,cleaned_genres
0,Les Particules Elementaires,Michel Houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires,michel houellebecq,le particules elementaires michel houellebecq ...,"[-0.22314462, 0.42006478, -0.28728843, -0.1840...","[-0.3414879, -0.07702848, -0.051483676, -0.296...","[0.11841526, 0.20468897, -0.028159915, -0.2183...","[-0.21087638, 0.38165882, -0.00066206243, -0.3...",fiction france literature novels french litera...


In [27]:
# def compute_combined_features(df, column_type):
#     #Computing TF-IDF Features
#     tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
#     tfidf_features = tfidf_vectorizer.fit_transform(df['new_combined'])

#     #Concatenating BERT and TF-IDF Features
#     bert_features = np.stack(df['combined_embeddings'].to_numpy())  # Use stack for an array of arrays
#     combined_features = hstack([bert_features, tfidf_features])

#     return combined_features, tfidf_vectorizer

In [99]:
def cosine_sims(df, book_title):
    # if book_title not in df['Title'].values:
    #     print(f"Query title '{book_title}' not found in the dataset.")
    #     return df
    
# try creating new_cobijned with clean_text features for tfidf
    
    # all_titles = df['Title'].values
    all_titles = df['Genres'].values
    np.append(all_titles, (np.array([book_title])))
    all_titles = all_titles.astype("U")

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(all_titles)

    # Convert the TF-IDF matrix to CSR format
    tfidf_matrix_csr = sp.csr_matrix(tfidf_matrix)

    # Get the index of the query title
    # query_index = df.index[df['Title'] == book_title].tolist()[0]

    # Compute cosine similarity between the query title and all titles
    query_vector = tfidf_matrix_csr[-1]
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix_csr).flatten()


    # cosine_similarities = cosine_similarity(query_index, tfidf_matrix_csr).flatten()

    # Add cosine similarity values to a new column in the DataFrame
    df['Cosine Similarity'] = cosine_similarities

    return df

In [28]:
# NEW FEATURE COMPUTATION

def compute_combined_features(df, column_type, embeddings):
    #Computing TF-IDF Features
    tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
    tfidf_features = tfidf_vectorizer.fit_transform(df[column_type])

    #Concatenating BERT and TF-IDF Features
    bert_features = np.stack(df[embeddings].to_numpy())  # Use stack for an array of arrays
    combined_features = hstack([bert_features, tfidf_features])

    return combined_features, tfidf_vectorizer

In [130]:
# NEW COMBINED

def content_based_recommendations(query, df, type = "combined", n_recommendations=5):

    if 'BookID' in df.columns:
        df = gbx_content
    else:
        df = bx_content

    if type.lower() == "combined":
        column_type = "new_combined"
        embeddings = "combined_embeddings"
    elif type.lower() == "title":
        column_type = "Title"
        embeddings = "title_embeddings"
    elif type.lower() == "author":
        column_type = "Author"
        embeddings = "authors_embeddings"
    elif type.lower() == "genres":
        column_type = "cleaned_genres"
        embeddings = "genres_embeddings"

    # cleaned_query = clean_text(query)
    # Compute combined features for the specified DataFrame
    combined_features, tfidf_vectorizer = compute_combined_features(df, column_type, embeddings)

    # Use KNN for Recommendations
    knn_model = NearestNeighbors(n_neighbors=n_recommendations, metric='cosine')
    knn_model.fit(combined_features)

    query_embedding = get_bert_embeddings_batch([query])[0].astype(np.float64)
    query_tfidf = tfidf_vectorizer.transform([query])
    query_combined = hstack([np.array([query_embedding]), query_tfidf])

    distances, indices = knn_model.kneighbors(query_combined, n_neighbors=n_recommendations*10)

    # Flatten indices and fetch recommendations
    recommendations = df.iloc[indices.flatten()]

    # Ensure unique recommendations based on Title and Author, excluding the query itself
    unique_titles = set()
    unique_recommendations = []
    for _, row in recommendations.iterrows():
        unique_key = (row['Title'].lower(), row['Author'].lower())
        if unique_key not in unique_titles and row['Title'].lower() != query.lower():
            unique_titles.add(unique_key)
            unique_recommendations.append(row)
            if len(unique_recommendations) >= n_recommendations:
                break    
    print("Your query:-")
    print("Query:", query)
    # print("Title:", row['Title'])
    # print("Author:", row['Author'])
    # print("Genres:", row['Genres'])
    # print(df.loc[df['Title'] == query, 'Genres'].values)
    # return pd.DataFrame(unique_recommendations, columns=['Title', 'Author', 'Genres']).reset_index(drop=True)

    combined_recs = pd.DataFrame(unique_recommendations, columns=['Title', 'Author', 'Genres']).reset_index(drop=True)


        # combined_recs = pd.concat([content_recs, cf_recs]).drop_duplicates(subset=['Title', 'Author'])
    # print(combined_recs)
    

    combined_recommendations = cosine_sims(combined_recs, book_title=query)  # Calculate cosine similarity
    sorted_recommendations = combined_recommendations.sort_values(by='Cosine Similarity', ascending=False)
    top_recommendations = sorted_recommendations.head(n_recommendations)
    final_recs = top_recommendations.loc[:, ['Title', 'Author', 'Genres']]
    return final_recs

    # return pd.DataFrame(unique_recommendations)

In [131]:
content_based_recommendations("the kite runner", df=gbx_content, type="genres", n_recommendations=5)

Your query:-
Query: the kite runner


,Title,Author,Genres
4,"Brown Bear, Brown Bear, What Do You See?","Bill Martin Jr., Eric Carle","Picture Books, Childrens, Animals, Fiction, Cl..."
0,Is Your Mama A Llama?,"Deborah Guarino, Steven Kellogg","Picture Books, Childrens, Animals, Storytime, ..."
1,"Llama, Llama Red Pajama",Anna Dewdney,"Picture Books, Childrens, Animals, Storytime, ..."
3,El Zorro,"Isabel Allende, Margaret Sayers Peden","Fiction, Graphic Novels, Poetry, Childrens, Ph..."
2,"1,000 Places to See Before You Die",Patricia Schultz,"Travel, Nonfiction, Reference, Adventure, Hist..."


In [123]:
# def content_based_recommendations(query, df, top_k=10):

#     if 'BookID' in df.columns:
#         df = gbx_content
#     else:
#         df = bx_content

#     # cleaned_query = clean_text(query)
#     # Compute combined features for the specified DataFrame
#     combined_features, tfidf_vectorizer = compute_combined_features(df)

#     # Use KNN for Recommendations
#     knn_model = NearestNeighbors(n_neighbors=top_k, metric='cosine')
#     knn_model.fit(combined_features)

#     query_embedding = get_bert_embeddings_batch([query])[0].astype(np.float64)
#     query_tfidf = tfidf_vectorizer.transform([query])
#     query_combined = hstack([np.array([query_embedding]), query_tfidf])

#     distances, indices = knn_model.kneighbors(query_combined, n_neighbors=top_k*10)

#     # Flatten indices and fetch recommendations
#     recommendations = df.iloc[indices.flatten()]

#     # Ensure unique recommendations based on Title and Author, excluding the query itself
#     unique_titles = set()
#     unique_recommendations = []
#     for _, row in recommendations.iterrows():
#         unique_key = (row['Title'].lower(), row['Author'].lower())
#         if unique_key not in unique_titles and row['Title'].lower() != query.lower():
#             unique_titles.add(unique_key)
#             unique_recommendations.append(row)
#             if len(unique_recommendations) >= top_k:
#                 break    
#     return pd.DataFrame(unique_recommendations, columns=['Title', 'Author', 'Genres']).reset_index(drop=True)

#     # return pd.DataFrame(unique_recommendations)

In [56]:
content_based_recommendations("John Lawrence", bx_content, "combined", top_k=5)

,Title,Author,Genres
0,Next to a Letter from Home: Major Glenn Miller...,Geoffrey Butcher,Music
1,The Collected Poems of Weldon Kees,Donald Justice,"Poetry, Literature"
2,The Group of Seven and Tom Thomson: An Introdu...,Anne Newlands,Art
3,Prince What-A-Mess,Frank Muir,Animals
4,"Inheritance (Western Lovers, 9)",Joleen Daniels,Harlequin


In [60]:
content_based_recommendations("Wuthering Heights", gbx_content, 'combined', top_k=5)


,Title,Author,Genres
0,Coming Home,Rosamunde Pilcher,"Comics, Graphic Novels, Marvel, Spider Man, Co..."
1,Annihilation,Jeff VanderMeer,"Science Fiction, Fiction, Horror, Fantasy, Mys..."
2,Graduation Day,Joelle Charbonneau,"Dystopia, Young Adult, Science Fiction, Fictio..."
3,When Beauty Tamed the Beast,Eloisa James,"Historical Romance, Romance, Historical, Histo..."
4,The Stars My Destination,"Alfred Bester, Neil Gaiman","Science Fiction, Fiction, Classics, Cyberpunk,..."


In [61]:
content_based_recommendations("The Art of War", gbx_content, type = "combined", top_k=5)

,Title,Author,Genres
0,The Brooklyn Follies,Paul Auster,"Fiction, Contemporary, American, New York, Nov..."
1,Tropic of Capricorn,Henry Miller,"Fiction, Classics, Literature, Novels, America..."
2,Nine Parts of Desire: The Hidden World of Isla...,Geraldine Brooks,"Nonfiction, Religion, Islam, History, Feminism..."
3,The 20th Century Art Book,Phaidon Press,"Art, Nonfiction, Reference, Art History, Histo..."
4,The Discoverers: A History of Man's Search to ...,Daniel J. Boorstin,"History, Nonfiction, Science, Philosophy, Worl..."


In [94]:
content_based_recommendations("Harry Potter and the Goblet of Fire", bx_content, type="title", n_recommendations=5)

Your query:-
Query: Harry Potter and the Goblet of Fire


,Title,Author,Genres,Cosine Similarity
4,The Illuminatus! Trilogy: The Eye in the Pyram...,Robert Shea,"Fiction, Science Fiction, Fantasy, Humor, Phil...",1.000000
3,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Fantasy,0.174034
0,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,"Fantasy, Young Adult, Fiction, Magic, Children...",0.163998
1,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,"Fantasy, Fiction, Young Adult, Magic, Children...",0.163998
2,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,"Fantasy, Fiction, Young Adult, Magic, Children...",0.163998


In [100]:
content_based_recommendations("Harry Potter and the Goblet of Fire", bx_content, type="title", n_recommendations=5)

Your query:-
Query: Harry Potter and the Goblet of Fire


,Title,Author,Genres,Cosine Similarity
4,The Illuminatus! Trilogy: The Eye in the Pyram...,Robert Shea,"Fiction, Science Fiction, Fantasy, Humor, Phil...",1.000000
3,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Fantasy,0.174034
0,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,"Fantasy, Young Adult, Fiction, Magic, Children...",0.163998
1,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,"Fantasy, Fiction, Young Adult, Magic, Children...",0.163998
2,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,"Fantasy, Fiction, Young Adult, Magic, Children...",0.163998


# Collaborative-filtering Model

##### Memory-based item-similarity CF using kNN

In [30]:
bx_cols_to_drop = ['ISBN', 'title_embeddings', 'combined_embeddings', 'genres_embeddings', 'combined_cluster_label', 'genre_cluster_label']
bx_cf = bx_cleaned.drop(columns=bx_cols_to_drop)
bx_cf.head(1)

,UserID,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined,cleaned_genres,title_genres_combined,authors_embeddings
0,276733,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...,fiction france literature novels french litera...,le particules elementaires fiction france lite...,"[-0.3414879, -0.07702848, -0.051483676, -0.296..."


In [31]:
gbx_cols_to_drop = ['id', 'ISBN', 'title_embeddings', 'combined_embeddings', 'genres_embeddings', 'combined_cluster_label', 'genre_cluster_label']
gbx_cf = gbx_cleaned.drop(columns=gbx_cols_to_drop)
gbx_cf.head(1)

,BookID,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,cleaned_genres,title_genres_combined,authors_embeddings
0,2767052,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,young adult fiction fantasy dystopia science f...,hunger game young adult fiction fantasy dystop...,"[-0.12804408, 0.11276019, -0.25651056, -0.0526..."


In [ ]:
# user_item_matrix_bx = bx_cf.pivot_table(index='UserID', columns='cleaned_title', values='User_Rating').fillna(0)

In [24]:
# user_item_matrix_gbx = gbx_cf.pivot_table(index='UserID', columns='cleaned_title', values='User_Rating').fillna(0)

In [25]:
# genres_user_item_matrix_gbx = gbx_cf.pivot_table(index='UserID', columns='cleaned_genres', values='User_Rating').fillna(0)

In [ ]:
# genres_user_item_matrix_bx = bx_cf.pivot_table(index='UserID', columns='cleaned_genres', values='User_Rating').fillna(0)

In [ ]:
# author_user_item_matrix_bx = bx_cf.pivot_table(index='UserID', columns='cleaned_author', values='User_Rating').fillna(0)

In [ ]:
# author_user_item_matrix_gbx = gbx_cf.pivot_table(index='UserID', columns='cleaned_author', values='User_Rating').fillna(0)

In [203]:
# combined_user_item_matrix_gbx = gbx_cf.pivot_table(index='UserID', columns='title_genres_combined', values='User_Rating').fillna(0)

In [204]:
# combined_user_item_matrix_bx = bx_cf.pivot_table(index='UserID', columns='title_genres_combined', values='User_Rating').fillna(0)

In [116]:
# user_item_matrix_bx.to_csv('user_item_matrix_bx.csv')

In [117]:
# user_item_matrix_gbx.to_csv('user_item_matrix_gbx.csv')

In [118]:
# genres_user_item_matrix_gbx.to_csv('genres_user_item_matrix_gbx.csv')

In [119]:
# genres_user_item_matrix_bx.to_csv('genres_user_item_matrix_bx.csv')

In [32]:
user_item_matrix_bx = pd.read_csv('user_item_matrix_bx.csv', index_col=0)

In [33]:
genres_user_item_matrix_gbx = pd.read_csv('genres_user_item_matrix_gbx.csv', index_col=0)

In [34]:
genres_user_item_matrix_bx = pd.read_csv('genres_user_item_matrix_bx.csv', index_col=0)

In [35]:
user_item_matrix_gbx = pd.read_csv('user_item_matrix_gbx.csv', index_col=0)

In [36]:
def get_popular_books(df, n=5):
    book_popularity = df.groupby('cleaned_title')['User_Rating'].count().sort_values(ascending=False).head(n)
    # book_popularity = df.groupby('title_genres_combined')['User_Rating'].count().sort_values(ascending=False).head(n)

    popular_books_details = df[df['cleaned_title'].isin(book_popularity.index)][['Title', 'Author', 'Genres']].drop_duplicates().head(n)
    # popular_books_details = df[df['title_genres_combined'].isin(book_popularity.index)][['Title', 'Author', 'Genres']].drop_duplicates().head(n)
    return popular_books_details

In [132]:
# def item_collaborative_recommendations(book_title, df, n_recommendations=5):
#     cleaned_book_title = clean_text(book_title)
    
#     # Assuming user_item_matrix is predefined and contains your user-item interaction data
#     if 'BookID' in df.columns:
#         df = gbx_cf
#         user_item_matrix = user_item_matrix_gbx
#     else:
#         df = bx_cf
#         user_item_matrix = user_item_matrix_bx
    
#     if cleaned_book_title not in user_item_matrix.columns:
#     # if cleaned_book_title not in df['cleaned_title'].values:
#         print("Book title not found. Recommending popular books instead.")
#         return get_popular_books(df, n_recommendations)
    
#     # Initialize and train kNN model
#     model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
#     model_knn.fit(user_item_matrix.T.values)  # Transpose to get item-feature matrix

#     # Locate the index of the book title
#     book_index = list(user_item_matrix.columns).index(cleaned_book_title)
#     distances, indices = model_knn.kneighbors(user_item_matrix.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=n_recommendations*2)

#     # Fetch indices for the recommended books, excluding the book itself
#     recommended_indices = indices.flatten()[1:]

#     # Fetch book titles for the recommended indices
#     recommended_books_titles = [user_item_matrix.columns[i] for i in recommended_indices]
    
#     # Fetch the details for the recommended books
#     recommended_books_details = df[df['cleaned_title'].isin(recommended_books_titles)].drop_duplicates(subset=['cleaned_title'])
#     recommended_books_details = recommended_books_details[['Title', 'Author', 'Genres']].head(n_recommendations)
    
#     # print("Your query:-")
#     # print("Query:", book_title)
#     # print("Title:", row['Title'])
#     # print("Author:", row['Author'])
#     # print("Genres:", df[df['Genres']])
    
#     return recommended_books_details

In [37]:
book_title = "Perfect"
item_collaborative_recommendations(book_title, bx_cf, n_recommendations=5)

Book title not found. Recommending popular books instead.


,Title,Author,Genres
72,The Lovely Bones: A Novel,Alice Sebold,"Fiction, Mystery, Young Adult, Contemporary, F..."
82,Wild Animus,Rich Shapero,"Fiction, Fantasy, Novels, Adult Fiction, Audio..."
107,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Fantasy
295,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,"Fiction, Chick Lit, Contemporary, Adult Fictio..."
368,The Rainmaker,John Grisham,"Fiction, Thriller, Mystery, Legal Thriller, Cr..."


In [38]:
book_title = "The Hunger Games"
item_collaborative_recommendations(book_title, bx_cf, n_recommendations=5)

Book title not found. Recommending popular books instead.


,Title,Author,Genres
72,The Lovely Bones: A Novel,Alice Sebold,"Fiction, Mystery, Young Adult, Contemporary, F..."
82,Wild Animus,Rich Shapero,"Fiction, Fantasy, Novels, Adult Fiction, Audio..."
107,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Fantasy
295,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,"Fiction, Chick Lit, Contemporary, Adult Fictio..."
368,The Rainmaker,John Grisham,"Fiction, Thriller, Mystery, Legal Thriller, Cr..."


In [39]:
book_title = "Twilight"
item_collaborative_recommendations(book_title, bx_cf, n_recommendations=5)

,Title,Author,Genres
573,FAST LANES (Washington Square),Jayne Anne Phillips,"Short Stories, Fiction, Literary Fiction"
12321,The Falling Nun: And Other Stories,Pamela Rafael Berkman,"Short Stories, Fiction"
12397,The Book Group Book: A Thoughtful Guide to For...,Ellen Slezak,"Nonfiction, Books About Books"
19017,The Knockout Artist,Harry Crews,"Fiction, Southern, Novels, American, Literatur..."
20119,The It-Doesn'T-Matter Suit,Sylvia Plath,"Fiction, Childrens, Poetry, Short Stories, Pic..."


In [40]:
book_title = "Twilight"
item_collaborative_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
261298,Just After Sunset,Stephen King,"Horror, Short Stories, Fiction, Thriller, Fant..."
397675,Rising storm,Erin Hunter,"Fiction, Thriller, Military Fiction, War, Acti..."
412825,A Dangerous Path,Erin Hunter,"Fantasy, Animals, Fiction, Middle Grade, Child..."
427057,Midnight,Erin Hunter,"Fiction, Fantasy, Contemporary, Audiobook, Mag..."
447412,Ninth Key,Meg Cabot,"Young Adult, Fantasy, Paranormal, Romance, Gho..."


In [41]:
book_title = "The Alchemist"
item_collaborative_recommendations(book_title, bx_cf, n_recommendations=5)

,Title,Author,Genres
4440,Walt Whitman: The Complete Poems (Penguin Clas...,Francis Murphy,"Poetry, Classics, Fiction, Literature, America..."
4441,Vivian Lives,Sherrie Krantz,Chick Lit
4454,MAKING YOUR OWN DAYS : THE PLEASURES OF READIN...,Kenneth Koch,"Poetry, Writing, Nonfiction, Literary Criticis..."
4457,Writing for Dollars,John Mccollister,"Writing, Reference, Business, Nonfiction"
4458,Northern Exposure: The Official Publication of...,Louis Chunovic,"Nonfiction, Tv, Pop Culture"


In [42]:
book_title = "The Shining"
item_collaborative_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
1000,The Kite Runner,Khaled Hosseini,"Fiction, Historical Fiction, Contemporary, Nov..."
1500,MÃ¤n som hatar kvinnor,"Stieg Larsson, Reg Keeland","Fiction, Mystery, Thriller, Crime, Mystery Thr..."
1800,The Fellowship of the Ring,J.R.R. Tolkien,"Fantasy, Classics, Fiction, Adventure, High Fa..."
3100,Memoirs of a Geisha,Arthur Golden,"Fiction, Historical Fiction, Romance, Classics..."
3300,O Alquimista,"Paulo Coelho, Alan R. Clarke","Fiction, Fantasy, Philosophy, Novels, Spiritua..."


In [44]:
book_title = "The Adventures of Sherlock Holmes"
item_collaborative_recommendations(book_title, bx_cf, n_recommendations=5)

,Title,Author,Genres
12472,Singing Bird,Roisin McAuley,"Fiction, Mystery, Contemporary, Australia, Lit..."
20133,Vivienne,Richard Hoyt,"Fashion, Biography, Nonfiction, Art, Memoir, B..."
23539,MOVIOLA,Garson Kanin,"Fiction, Historical Fiction, Film"
23837,Founding Fathers: Brief Lives of the Framers o...,M. E. Bradford,"History, American History"
27189,Black Valentine,Patricia Sargent,"Romance, New Adult, Music, Contemporary Romanc..."


In [43]:
book_title = "The Adventures of Sherlock Holmes"
item_collaborative_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
18500,A Christmas Carol,Charles Dickens,"Fiction, Christmas, Classics, Fantasy, Audiobo..."
27000,Treasure Island,Robert Louis Stevenson,"Classics, Fiction, Adventure, Historical Ficti..."
31900,The Wonderful Wizard of Oz,"L. Frank Baum, W.W. Denslow","Fantasy, Fiction, Childrens, Classics, Young A..."
54197,David Copperfield,"Charles Dickens, Jeremy Tambling","Fiction, Classics, Literature, Historical Fict..."
56797,The Hound of the Baskervilles,"Arthur Conan Doyle, Anne Perry","Mystery, Fiction, Crime, Classics, Mystery Thr..."


In [139]:
def collaborative_recommendations(query, df, type="Title", n_recommendations=5):
    
    cleaned_book_title = clean_text(query)

    if type.lower() == "title":

        if 'BookID' in df.columns:
            df = gbx_cf
            user_item_matrix = user_item_matrix_gbx
        else:
            df = bx_cf
            user_item_matrix = user_item_matrix_bx

        if cleaned_book_title not in user_item_matrix.columns:
            print("Book title not found. Recommending popular books instead.")
            return get_popular_books(df, n_recommendations)
        
        model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
        model_knn.fit(user_item_matrix.T.values)  # Transpose to get item-feature matrix

        book_index = list(user_item_matrix.columns).index(cleaned_book_title)
        distances, indices = model_knn.kneighbors(user_item_matrix.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=n_recommendations*2)

        recommended_indices = indices.flatten()[1:]
        recommended_books_titles = [user_item_matrix.columns[i] for i in recommended_indices]

        recommended_books_details = df[df['cleaned_title'].isin(recommended_books_titles)].drop_duplicates(subset=['cleaned_title'])
        recommended_books_details = recommended_books_details[['Title', 'Author', 'Genres']].head(n_recommendations)


    elif type.lower() == "genres":
        if 'BookID' in df.columns:
            df = gbx_cf
            user_item_matrix = genres_user_item_matrix_gbx
        else:
            df = bx_cf
            user_item_matrix = genres_user_item_matrix_bx

        if cleaned_book_title not in df['cleaned_title'].values:
            print("Book title not found. Recommending popular books instead.")
            return get_popular_books(df, n_recommendations)
        
        book_genres_set = set(df.loc[df['cleaned_title'] == cleaned_book_title, 'cleaned_genres'].iloc[0].lower().split())
        # print(book_genres_set)

        relevant_columns = []
        for col in user_item_matrix.columns:
            if set(col.split()) & book_genres_set:
                relevant_columns.append(col)

        # print(relevant_columns)

        if not relevant_columns:
            print("Matching genres not found in the matrix. Recommending popular books instead.")
            return get_popular_books(df, n_recommendations)

        model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
        model_knn.fit(user_item_matrix[relevant_columns].values.T)  

        query_vector = user_item_matrix[relevant_columns].mean(axis=1).values.reshape(1, -1)
        distances, indices = model_knn.kneighbors(query_vector, n_neighbors=n_recommendations*2)

        recommended_indices = indices.flatten()[1:]
        recommended_genres = [user_item_matrix.columns[i] for i in recommended_indices]

        recommended_books = df[df['cleaned_genres'].apply(lambda x: any(genre in x for genre in recommended_genres))].drop_duplicates(subset=['cleaned_title'])
        recommended_books_details = recommended_books[['Title', 'Author', 'Genres']].head(n_recommendations)

        # final_recs = recommended_books_details.loc[:, ['Title', 'Author', 'Genres']]
    combined_recommendations = cosine_sims(recommended_books_details, book_title=query)  # Calculate cosine similarity
    sorted_recommendations = combined_recommendations.sort_values(by='Cosine Similarity', ascending=False)
    top_recommendations = sorted_recommendations.head(n_recommendations)
    final_recs = top_recommendations.loc[:, ['Title', 'Author', 'Genres']]
    return final_recs

    
    # return recommended_books_details

In [56]:
# from sklearn.neighbors import NearestNeighbors

# def collaborative_recommendations(query, df, type="Title", n_recommendations=5):
#     cleaned_book_title = clean_text(query)

#     if 'BookID' in df.columns:
#         user_item_matrix = user_item_matrix_gbx if type.lower() == "title" else genres_user_item_matrix_gbx
#     else:
#         user_item_matrix = user_item_matrix_bx if type.lower() == "title" else genres_user_item_matrix_bx

#     if type.lower() == "title":
#         if cleaned_book_title not in user_item_matrix.columns:
#             print("Book title not found. Recommending popular books instead.")
#             return get_popular_books(df, n_recommendations)
        
#         model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
#         model_knn.fit(user_item_matrix.T.values)  

#         book_index = list(user_item_matrix.columns).index(cleaned_book_title)
#         distances, indices = model_knn.kneighbors(user_item_matrix.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=n_recommendations*2)

#         recommended_indices = indices.flatten()[1:]
#         recommended_books_titles = [user_item_matrix.columns[i] for i in recommended_indices]

#         recommended_books_details = df[df['cleaned_title'].isin(recommended_books_titles)].drop_duplicates(subset=['cleaned_title'])
#         recommended_books_details = recommended_books_details[['Title', 'Author', 'Genres']].head(n_recommendations)

#     elif type.lower() == "genres":
#         if cleaned_book_title not in df['cleaned_title'].values:
#             print("Book title not found. Recommending popular books instead.")
#             return get_popular_books(df, n_recommendations)
        
#         book_genres_set = set(df.loc[df['cleaned_title'] == cleaned_book_title, 'cleaned_genres'].iloc[0].lower().split())
#         relevant_columns = [col for col in user_item_matrix.columns if set(col.split()) & book_genres_set]

#         if not relevant_columns:
#             print("Matching genres not found in the matrix. Recommending popular books instead.")
#             return get_popular_books(df, n_recommendations)

#         model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
#         model_knn.fit(user_item_matrix[relevant_columns].values.T)  

#         query_vector = user_item_matrix[relevant_columns].mean(axis=1).values.reshape(1, -1)
#         distances, indices = model_knn.kneighbors(query_vector, n_neighbors=n_recommendations*2)

#         recommended_indices = indices.flatten()[1:]
#         recommended_genres = [user_item_matrix.columns[i] for i in recommended_indices]

#         recommended_books = df[df['cleaned_genres'].apply(lambda x: any(genre in x for genre in recommended_genres))].drop_duplicates(subset=['cleaned_title'])
#         recommended_books_details = recommended_books[['Title', 'Author', 'Genres']].head(n_recommendations)

#     return recommended_books_details


In [135]:
book_title = "The Adventures of sherlock holmes"
collaborative_recommendations(book_title, df=gbx_cf, type="Genres", n_recommendations=5)

,Title,Author,Genres
390803,How to Tell If Your Cat Is Plotting to Kill You,"Matthew Inman, The Oatmeal","Humor, Graphic Novels, Comics, Cats, Nonfictio..."
245116,The Four Loves,C.S. Lewis,"Christian, Nonfiction, Theology, Religion, Phi..."
19300,Die Verwandlung,"Franz Kafka, Stanley Corngold","Classics, Fiction, Fantasy, Philosophy, Litera..."
37199,The Tell-Tale Heart,Edgar Allan Poe,"Classics, Horror, Fiction, Short Stories, Poet..."
256502,Ariel,Sylvia Plath,"Classics, Feminism, Fiction, Poetry, 20th Cent..."


In [140]:
book_title = "The Adventures of sherlock holmes"
collaborative_recommendations(book_title, df=gbx_cf, type="Title", n_recommendations=5)

,Title,Author,Genres
56797,The Hound of the Baskervilles,"Arthur Conan Doyle, Anne Perry","Mystery, Fiction, Crime, Classics, Mystery Thr..."
18500,A Christmas Carol,Charles Dickens,"Fiction, Christmas, Classics, Fantasy, Audiobo..."
27000,Treasure Island,Robert Louis Stevenson,"Classics, Fiction, Adventure, Historical Ficti..."
54197,David Copperfield,"Charles Dickens, Jeremy Tambling","Fiction, Classics, Literature, Historical Fict..."
31900,The Wonderful Wizard of Oz,"L. Frank Baum, W.W. Denslow","Fantasy, Fiction, Childrens, Classics, Young A..."


In [58]:
book_title = "The Adventures of sherlock holmes"
collaborative_recommendations(book_title, df=bx_cf, type="Title", n_recommendations=5)

,Title,Author,Genres
12472,Singing Bird,Roisin McAuley,"Fiction, Mystery, Contemporary, Australia, Lit..."
20133,Vivienne,Richard Hoyt,"Fashion, Biography, Nonfiction, Art, Memoir, B..."
23539,MOVIOLA,Garson Kanin,"Fiction, Historical Fiction, Film"
23837,Founding Fathers: Brief Lives of the Framers o...,M. E. Bradford,"History, American History"
27189,Black Valentine,Patricia Sargent,"Romance, New Adult, Music, Contemporary Romanc..."


In [54]:
book_title = "The Adventures of sherlock holmes"
collaborative_recommendations(book_title, df=bx_cf, type="Genres", n_recommendations=5)

,Title,Author,Genres
415,Charmed Circle,Carol Smith,"Biography, Nonfiction, History, Art, France, Q..."
615,The Remnant: On the Brink of Armageddon (Left ...,Tim LaHaye,"Christian Fiction, Christian, Fiction, Religio..."
3168,One Incredible Moment Celebrating The Majesty ...,Max Lucado,"Christmas, Christian, Nonfiction, Religion, Ho..."
6271,Le Rocher de Tanios,Amin Maalouf,"Fiction, Historical Fiction, France, Novels, L..."
9744,Tecniche di seduzione,Andrea De Carlo,"Italian Literature, Fiction, Novels, Italy"


In [55]:
book_title = "The Adventures of sherlock holmes"
collaborative_recommendations(book_title, df=bx_cf, type="Title", n_recommendations=5)

,Title,Author,Genres
12472,Singing Bird,Roisin McAuley,"Fiction, Mystery, Contemporary, Australia, Lit..."
20133,Vivienne,Richard Hoyt,"Fashion, Biography, Nonfiction, Art, Memoir, B..."
23539,MOVIOLA,Garson Kanin,"Fiction, Historical Fiction, Film"
23837,Founding Fathers: Brief Lives of the Framers o...,M. E. Bradford,"History, American History"
27189,Black Valentine,Patricia Sargent,"Romance, New Adult, Music, Contemporary Romanc..."


In [48]:
book_title = "Looking For Alaska"
collaborative_recommendations(book_title, df=gbx_cf, type="Title", n_recommendations=5)

,Title,Author,Genres
500,The Fault in Our Stars,John Green,"Young Adult, Fiction, Contemporary, Realistic ..."
1100,Divergent,Veronica Roth,"Young Adult, Dystopia, Fantasy, Fiction, Scien..."
6300,The Perks of Being a Wallflower,Stephen Chbosky,"Young Adult, Fiction, Coming Of Age, Classics,..."
8000,Paper Towns,John Green,"Young Adult, Fiction, Contemporary, Romance, M..."
8300,The Maze Runner,James Dashner,"Young Adult, Dystopia, Science Fiction, Fictio..."


In [49]:
book_title = "Looking For Alaska"
collaborative_recommendations(book_title, df=gbx_cf, type="Genres", n_recommendations=5)

,Title,Author,Genres
99997,90 Minutes in Heaven,"Don Piper, Cecil Murphey","Christian, Nonfiction, Religion, Spirituality,..."
159783,The Black Stallion,"Walter Farley, Keith Ward","Classics, Fiction, Childrens, Horses, Young Ad..."
302013,Les Fleurs du mal,"Charles Baudelaire, Richard Howard","Classics, France, Poetry, French Literature, F..."
308972,The Prodigal Daughter,Jeffrey Archer,"Fiction, Historical Fiction, Thriller, Drama, ..."
348590,Ben-Hur: A Tale of the Christ,Lew Wallace,"Classics, Historical Fiction, Fiction, Christi..."


In [50]:
book_title = "thirteen reasons why"
collaborative_recommendations(book_title, df=gbx_cf, type="title", n_recommendations=5)

,Title,Author,Genres
6900,Looking for Alaska,John Green,"Young Adult, Fiction, Contemporary, Romance, R..."
8000,Paper Towns,John Green,"Young Adult, Fiction, Contemporary, Romance, M..."
8300,The Maze Runner,James Dashner,"Young Adult, Dystopia, Science Fiction, Fictio..."
13600,If I Stay,Gayle Forman,"Young Adult, Romance, Contemporary, Fiction, R..."
16600,Beautiful Creatures,"Kami Garcia, Margaret Stohl","Fantasy, Young Adult, Paranormal, Romance, Fic..."


In [51]:
book_title = "thirteen reasons why"
collaborative_recommendations(book_title, df=gbx_cf, type="genres", n_recommendations=5)

,Title,Author,Genres
36099,The Strange Case of Dr Jekyll and Mr Hyde,"Robert Louis Stevenson, Vladimir Nabokov, Merv...","Classics, Horror, Fiction, Gothic, Science Fic..."
45498,Mere Christianity,C.S. Lewis,"Christian, Nonfiction, Religion, Theology, Chr..."
90897,The Scarlet Pimpernel,Emmuska Orczy,"Classics, Historical Fiction, Fiction, Adventu..."
114286,Mary Poppins,"P.L. Travers, Mary Shepard","Classics, Fantasy, Childrens, Fiction, Middle ..."
123585,The Strange Case of Dr. Jekyll and Mr. Hyde an...,"Robert Louis Stevenson, Robert Mighall","Classics, Horror, Fiction, Gothic, Science Fic..."


In [52]:
book_title = "A thousand splendid suns"
collaborative_recommendations(book_title, df=gbx_cf, type="genres", n_recommendations=5)

,Title,Author,Genres
77897,"Twelfth Night; or, What You Will",William Shakespeare,"Classics, Plays, Drama, Fiction, School, Roman..."
99697,Sycamore Row,John Grisham,"Fiction, Mystery, Thriller, Legal Thriller, Au..."
135284,"Go, Dog. Go!",P.D. Eastman,"Childrens, Picture Books, Fiction, Classics, A..."
149583,The Poky Little Puppy (a Little Golden Book),"Janette Sebring Lowrey, Gustaf Tenggren","Childrens, Picture Books, Fiction, Classics, A..."
150983,A Room of One's Own,Virginia Woolf,"Classics, Nonfiction, Feminism, Essays, Philos..."


In [53]:
book_title = "A thousand splendid suns"
collaborative_recommendations(book_title, df=gbx_cf, type="title", n_recommendations=5)

,Title,Author,Genres
1000,The Kite Runner,Khaled Hosseini,"Fiction, Historical Fiction, Contemporary, Nov..."
2900,The Help,Kathryn Stockett,"Fiction, Historical Fiction, Historical, Adult..."
3100,Memoirs of a Geisha,Arthur Golden,"Fiction, Historical Fiction, Romance, Classics..."
4300,Life of Pi,Yann Martel,"Fiction, Fantasy, Classics, Adventure, Magical..."
5300,The Secret Life of Bees,Sue Monk Kidd,"Fiction, Historical Fiction, Classics, Young A..."


# Hybrid-filtering Model

##### Using BERT embeddings and kNN

In [39]:
bx_cols = ['ISBN', 'combined_cluster_label', 'genre_cluster_label']
bx_hybrid = bx_cleaned.drop(columns=bx_cols)
bx_hybrid.head(1)

,UserID,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,cleaned_genres,title_genres_combined,authors_embeddings
0,276733,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...,"[-0.22314462, 0.42006478, -0.28728843, -0.1840...","[-0.21087638, 0.38165882, -0.00066206243, -0.3...","[0.11841526, 0.20468897, -0.028159915, -0.2183...",fiction france literature novels french litera...,le particules elementaires fiction france lite...,"[-0.3414879, -0.07702848, -0.051483676, -0.296..."


In [40]:
gbx_cols = ['id', 'ISBN', 'combined_cluster_label', 'genre_cluster_label']
gbx_hybrid = gbx_cleaned.drop(columns=bx_cols)
gbx_hybrid.head(1)

,id,BookID,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,cleaned_genres,title_genres_combined,authors_embeddings
0,1,2767052,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...",young adult fiction fantasy dystopia science f...,hunger game young adult fiction fantasy dystop...,"[-0.12804408, 0.11276019, -0.25651056, -0.0526..."


In [ ]:
item_similarity_matrix_bx = cosine_similarity(bx_title_embeddings_array)

In [ ]:
item_similarity_matrix_gbx = cosine_similarity(gbx_title_embeddings_array)

In [35]:
# def hybrid_bert_recommendations(book_title, df, n_recommendations=5):
#     # Construct item-item similarity matrix
#     if 'BookID' in df.columns:
#         item_similarity_matrix = user_item_matrix_gbx
#     else:
#         item_similarity_matrix = user_item_matrix_bx
#     # item_similarity_matrix = construct_item_similarity_matrix(df)
    
#     # Find the index of the book title in the dataframe
#     book_index = df[df['Title'] == book_title].index[0]
    
#     # Use kNN with item similarity matrix to find similar items
#     model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
#     model_knn.fit(item_similarity_matrix)
    
#     # Find nearest neighbors (similar items) for the specified book title
#     distances, indices = model_knn.kneighbors([item_similarity_matrix[book_index]], n_neighbors=n_recommendations+1)
    
#     # Exclude the first item (itself) and get top n recommendations
#     recommended_indices = indices.flatten()[1:]
#     recommended_books_details = df.iloc[recommended_indices][['Title', 'Author', 'Genres']]
    
#     return recommended_books_details



# Assuming user_item_matrix_gbx and user_item_matrix_bx are defined elsewhere

def hybrid_bert_recommendations(book_title, df, n_recommendations=5):
    # Determine which user-item matrix to use based on the DataFrame
    if 'BookID' in df.columns:
        user_item_matrix = user_item_matrix_gbx
    else:
        user_item_matrix = user_item_matrix_bx
    
    # Convert user-item matrix to item-item similarity matrix
    item_similarity_matrix = cosine_similarity(user_item_matrix.T)
    
    # Find the index of the book title in the dataframe
    book_index = df[df['Title'] == book_title].index[0]
    
    # Use kNN with item similarity matrix to find similar items
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
    model_knn.fit(item_similarity_matrix)
    
    # Find nearest neighbors (similar items) for the specified book title
    distances, indices = model_knn.kneighbors([item_similarity_matrix[book_index]], n_neighbors=n_recommendations+1)
    
    # Exclude the first item (itself) and get top n recommendations
    recommended_indices = indices.flatten()[1:]
    recommended_books_details = df.iloc[recommended_indices][['Title', 'Author', 'Genres']]
    
    return recommended_books_details


In [56]:
book_title = "Twilight"
hybrid_bert_recommendations(book_title, gbx_hybrid, n_recommendations=5)

,Title,Author,Genres
6689,"Frankenstein; or, The Modern Prometheus","Mary Wollstonecraft Shelley, Percy Bysshe Shel...","Horror, Fiction, Science Fiction, Gothic, Fant..."
1190,Divergent,Veronica Roth,"Young Adult, Dystopia, Fantasy, Fiction, Scien..."
1992,Mockingjay,Suzanne Collins,"Young Adult, Dystopia, Fiction, Fantasy, Scien..."
4200,The Notebook,Nicholas Sparks,"Romance, Fiction, Chick Lit, Contemporary, Adu..."
2072,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPrÃ©","Young Adult, Fiction, Magic, Childrens, Audiob..."


In [57]:
book_title = "Twilight"
hybrid_bert_recommendations(book_title, bx_hybrid, n_recommendations=5)

,Title,Author,Genres
9198,Wenn ein Reisender in einer Winternacht.,Italo Calvino,"Fiction, Classics, Italian Literature, Magical..."
6156,The Book of Eleanor,PAMELA KAUFMAN,"Historical Fiction, Fiction, Mystery, Historic..."
10204,Temptation,Dermot Bolger,"Romance, Dark, Contemporary, Contemporary Roma..."
10198,The Last Promise,Richard Paul Evans,"Graphic Novels, Comics, Fantasy, Manga, Young ..."
4642,Falcon,Emma Bull,"Mystery, Fiction, Classics, Crime, Noir, Detec..."


In [58]:
book_title = "The Shining"
hybrid_bert_recommendations(book_title, gbx_hybrid, n_recommendations=5)

,Title,Author,Genres
7253,The Devil Wears Prada,Lauren Weisberger,"Chick Lit, Fiction, Contemporary, Romance, Adu..."
1440,Het Achterhuis: Dagboekbrieven 14 juni 1942 - ...,"Anne Frank, Eleanor Roosevelt, B.M. Mooyaart-D...","Classics, Nonfiction, Biography, Memoir, Histo..."
1016,The Kite Runner,Khaled Hosseini,"Fiction, Historical Fiction, Contemporary, Nov..."
2889,Gone Girl,Gillian Flynn,"Fiction, Mystery, Thriller, Mystery Thriller, ..."
3278,Fifty Shades of Grey,E.L. James,"Romance, Fiction, Erotica, Adult, BDSM, Contem..."


# Hybrid-filtering Model

#### Using Cascade Hybrid approach

In [108]:


# def cosine_sim(df, book_title):
#     tfidf_vectorizer = TfidfVectorizer()
#     tfidf_matrix = tfidf_vectorizer.fit_transform(df['Title'].values.astype('U'))

#     # Convert the TF-IDF matrix to CSR format
#     tfidf_matrix_csr = sp.csr_matrix(tfidf_matrix)

#     # Get the index of the query title
#     query_index = df.index[df['Title'] == book_title].tolist()[0]

#     # Compute cosine similarity between the query title and all titles
#     query_vector = tfidf_matrix_csr[query_index]
#     cosine_similarities = cosine_similarity(query_vector, tfidf_matrix_csr).flatten()

#     # Add cosine similarity values to a new column in the DataFrame
#     df['Cosine Similarity'] = cosine_similarities

#     return df

def cosine_sim(df, book_title):
    # if book_title not in df['Title'].values:
    #     print(f"Query title '{book_title}' not found in the dataset.")
    #     return df
    
# try creating new_cobijned with clean_text features for tfidf
    
    # all_titles = df['Title'].values
    all_titles = df['Genres'].values
    np.append(all_titles, (np.array([book_title])))
    all_titles = all_titles.astype("U")

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(all_titles)

    # Convert the TF-IDF matrix to CSR format
    tfidf_matrix_csr = sp.csr_matrix(tfidf_matrix)

    # Get the index of the query title
    # query_index = df.index[df['Title'] == book_title].tolist()[0]

    # Compute cosine similarity between the query title and all titles
    query_vector = tfidf_matrix_csr[-1]
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix_csr).flatten()


    # cosine_similarities = cosine_similarity(query_index, tfidf_matrix_csr).flatten()

    # Add cosine similarity values to a new column in the DataFrame
    df['Cosine Similarity'] = cosine_similarities

    return df

In [33]:
print(type(bx_hybrid["Title"].values.astype("U")))

<class 'numpy.ndarray'>


In [34]:
print(type(np.array(["Twilight"]).astype("U") ) )

<class 'numpy.ndarray'>


In [141]:
def cascade_hybrid_recss(query, df, type="combined", n_recommendations=5):
    
    recs_list = []
    # combined_recommendations = pd.DataFrame()

    if 'BookID' in df.columns:
        content_df = gbx_content
        cf_df = gbx_cf
    else:
        content_df = bx_content
        cf_df = bx_cf
    
    if type.lower() == "title":
        cont_type = "title"
        cf_type = "title"
    elif type.lower() == "author":
        cont_type = "author"
        cf_type = "title"
    elif type.lower() == "genres":
        cont_type = "genres"
        cf_type = "genres"
    elif type.lower() == "combined":
        cont_type = "combined"
        cf_type = "title"


    # initial_recommendations = content_based_recommendations(query=book_title, df=content_df, new_type, n_recommendations=n_recommendations)
    initial_recommendations = content_based_recommendations(query=query, df=content_df, type=cont_type, n_recommendations=5)

    if initial_recommendations.empty:
        return get_popular_books(df, n=n_recommendations)

    if not initial_recommendations.empty:
        # combined_recommendations = pd.DataFrame()
        for index,row in initial_recommendations.iterrows():
            # print(initial_recommendations)
            title = row['Title']
            # return book_title


            cf_recommendations = collaborative_recommendations(query=title, df=cf_df, type=cf_type, n_recommendations=2)

            # cf_recommendations = collaborative_recommendations(query=query, df=content_df, type=new_type, n_recommendations=2)

            recs_list.append(cf_recommendations)
            # print(recs_list)
        combined_recommendations = pd.concat(recs_list, ignore_index=True)
    # print(combined_recommendations)
    # combined_recommendations = combined_recommendations[combined_recommendations['Title'] != book_title]
        
    #try injecting the book title row into the combined recs dataframe to preserve the index HEREEEEEE
    # print(type(combined_recommendations))
    # original_book_row = df.loc[df['Title'] == book_title]
    
    # combined_recommendations = combined_recommendations.append(original_book_row.iloc[0], ignore_index=False)

    combined_recommendations = cosine_sim(combined_recommendations, book_title=query)  # Calculate cosine similarity
    # print(combined_recommendations.columns)
    # print(combined_recommendations)
    sorted_recommendations = combined_recommendations.sort_values(by='Cosine Similarity', ascending=False)
    top_recommendations = sorted_recommendations.head(n_recommendations)
    # top_recommendations = combined_recommendations.head(n_recommendations)
    final_recs = top_recommendations.loc[:, ['Title', 'Author', 'Genres']]

    # return combined_recommendations
    # return top_recommendations
    return final_recs
    

In [81]:
cascade_hybrid_recss("Wuthering Heights", bx_hybrid, n_recommendations=5)

Your query:-
Query: Wuthering Heights
Genres: Classics, Fiction, Romance, Gothic, Historical Fiction, Literature, Historical
                                               Title            Author  \
0                         The Last Lighthouse Keeper   Alan Titchmarsh   
1                                   Animal Instincts   Alan Titchmarsh   
2  On the Side of the Angels (Virago Modern Class...      Betty Miller   
3                                       Jacob's Room    VIRGINIA WOOLF   
4                  Man's Search for Ultimate Meaning  Viktor E. Frankl   
5                           Prince and Other Dogs II        Libby Hall   
6                              Die Frau des Piloten.      Anita Shreve   
7                        Schicksal in Person/Nemesis   Agatha Christie   
8  The DREAMS OUR STUFF IS MADE OF: How Science F...   Thomas M. Disch   
9                            Finn Family Moomintroll      Tove Jansson   

                                              Genres  
0  Fi

,Title,Author,Genres,Cosine Similarity
9,Finn Family Moomintroll,Tove Jansson,"Fantasy, Childrens, Fiction, Classics, Finnish...",1.000000
3,Jacob's Room,VIRGINIA WOOLF,"Classics, Fiction, 20th Century, Novels, Briti...",0.354451
2,On the Side of the Angels (Virago Modern Class...,Betty Miller,"Fiction, British Literature, Classics, World W...",0.247697
0,The Last Lighthouse Keeper,Alan Titchmarsh,"Fiction, Romance, British Literature, Audioboo...",0.211842
7,Schicksal in Person/Nemesis,Agatha Christie,"Mystery, Fiction, Crime, Classics, Mystery Thr...",0.091986


In [63]:
#new one
cascade_hybrid_recss("Wuthering Heights", gbx_hybrid, n_recommendations=5)

Your query:-
Query: Wuthering Heights
Genres: Classics, Fiction, Romance, Gothic, Historical Fiction, Literature, Historical


,Title,Author,Genres,Cosine Similarity
9,Finn Family Moomintroll,Tove Jansson,"Fantasy, Childrens, Fiction, Classics, Finnish...",1.000000
3,Jacob's Room,VIRGINIA WOOLF,"Classics, Fiction, 20th Century, Novels, Briti...",0.354451
2,On the Side of the Angels (Virago Modern Class...,Betty Miller,"Fiction, British Literature, Classics, World W...",0.247697
0,The Last Lighthouse Keeper,Alan Titchmarsh,"Fiction, Romance, British Literature, Audioboo...",0.211842
7,Schicksal in Person/Nemesis,Agatha Christie,"Mystery, Fiction, Crime, Classics, Mystery Thr...",0.091986


In [83]:
cascade_hybrid_recss("The Hunger Games", bx_hybrid, type="Genres", n_recommendations=5)

Your query:-
Query: The Hunger Games
Genres: Star Wars, Science Fiction, Fiction, Fantasy, Adventure, Space Opera, Science Fiction Fantasy
                                               Title            Author  \
0                                Lost Queen of Egypt   Lucile Morrison   
1  Chicago: With the Chicago Tribune Articles Tha...   Maurine Watkins   
2   Harry Potter and the Chamber of Secrets (Book 2)     J. K. Rowling   
3  Harry Potter and the Prisoner of Azkaban (Book 3)     J. K. Rowling   
4                Principles of Instrumental Analysis  Douglas A. Skoog   
5                      The Oxford Companion to Chess      David Hooper   
6               The Cartoon Guide to the Environment      Larry Gonick   
7                                The Book of the Sun        Tom Folley   
8                             Han Solo at Stars' End             Daley   
9                           Empire Strikes Back Wars     Donald F Glut   

                                              

,Title,Author,Genres,Cosine Similarity
8,Han Solo at Stars' End,Daley,"Star Wars, Science Fiction, Fiction, Space Ope...",1.000000
9,Empire Strikes Back Wars,Donald F Glut,"Star Wars, Science Fiction, Fiction, Fantasy, ...",1.000000
7,The Book of the Sun,Tom Folley,"Fantasy, Science Fiction, Fiction, Science Fic...",0.664955
2,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,"Fantasy, Fiction, Young Adult, Magic, Children...",0.218845
3,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,"Fantasy, Fiction, Young Adult, Magic, Children...",0.218845


In [84]:
#new one
cascade_hybrid_recss("The Hunger Games", gbx_hybrid, type="Genres", n_recommendations=5)

Your query:-
Query: The Hunger Games
Genres: Fantasy, Graphic Novels, Comics, Fiction, Adult, Graphic Novels Comics, Adventure
                                 Title  \
0                            Aftermath   
1            Star Wars: Darth Plagueis   
2                   Green Eggs and Ham   
3               The Return of the King   
4                To Kill a Mockingbird   
5                 Nineteen Eighty-Four   
6           The Sandman: A Game of You   
7  The Sandman: Fables and Reflections   
8                     Prince of Thorns   
9                   The Blinding Knife   

                                              Author  \
0                                       Chuck Wendig   
1                                       James Luceno   
2                         Dr. Seuss, ××× × ×××¨   
3                                     J.R.R. Tolkien   
4                                         Harper Lee   
5          George Orwell, Erich Fromm, CelÃ¢l Ãster   
6  Neil Gaiman, Sh

,Title,Author,Genres,Cosine Similarity
9,The Blinding Knife,Brent Weeks,"Fantasy, Fiction, Epic Fantasy, High Fantasy, ...",1.000000
3,The Return of the King,J.R.R. Tolkien,"Fantasy, Fiction, Adventure, Classics, High Fa...",0.684166
8,Prince of Thorns,Mark Lawrence,"Fantasy, Fiction, Dark Fantasy, Epic Fantasy, ...",0.580486
1,Star Wars: Darth Plagueis,James Luceno,"Star Wars, Science Fiction, Fiction, Fantasy, ...",0.244153
2,Green Eggs and Ham,"Dr. Seuss, ××× × ×××¨","Childrens, Picture Books, Classics, Fiction, P...",0.141021


In [87]:
#new one
cascade_hybrid_recss("The Kite Runner", gbx_hybrid, type="Genres", n_recommendations=5)

Your query:-
Query: The Kite Runner
Genres: Fantasy, Young Adult, Fiction, Magic, Adventure, Young Adult Fantasy, Romance


,Title,Author,Genres,Cosine Similarity
9,"Wild Magic (Immortals, #1)",Tamora Pierce,"Fantasy, Young Adult, Fiction, Magic, Young Ad...",1.000000
8,Lioness Rampant,Tamora Pierce,"Fantasy, Young Adult, Fiction, Magic, Young Ad...",0.844421
5,Prince Caspian: The Return to Narnia,C.S. Lewis,"Fantasy, Fiction, Classics, Young Adult, Child...",0.481540
4,The magician's nephew,C.S. Lewis,"Fantasy, Fiction, Young Adult, Childrens, Midd...",0.455649
1,"Oh, the Places You'll Go!",Dr. Seuss,"Childrens, Picture Books, Fiction, Classics, P...",0.382951


In [96]:
cascade_hybrid_recss("The Kite Runner", gbx_hybrid, type="combined", n_recommendations=5)

Your query:-
Query: The Kite Runner
Genres: Classics, Plays, Fiction, Drama, Theatre, 19th Century, Humor


,Title,Author,Genres,Cosine Similarity
9,The Canterville Ghost,"Oscar Wilde, Inga Moore","Classics, Fiction, Short Stories, Horror, Fant...",1.000000
7,Put Me in the Zoo,Robert Lopshire,"Picture Books, Childrens, Fiction, Animals, Cl...",0.215139
8,An Ideal Husband,Oscar Wilde,"Classics, Plays, Fiction, Drama, Humor, 19th C...",0.187767
6,The Cat in the Hat and Other Dr. Seuss Favorites,"Dr. Seuss, Various","Childrens, Picture Books, Classics, Fiction, P...",0.179077
1,The House at Pooh Corner,"A.A. Milne, Ernest H. Shepard","Classics, Childrens, Fiction, Fantasy, Animals...",0.176937


In [100]:
cascade_hybrid_recss("The Kite Runner", gbx_hybrid, type="combined", n_recommendations=5)

Your query:-
Query: The Kite Runner
Genres: Classics, Plays, Fiction, Drama, Theatre, 19th Century, Humor


,Title,Author,Genres,Cosine Similarity
9,The Canterville Ghost,"Oscar Wilde, Inga Moore","Classics, Fiction, Short Stories, Horror, Fant...",1.000000
7,Put Me in the Zoo,Robert Lopshire,"Picture Books, Childrens, Fiction, Animals, Cl...",0.215139
8,An Ideal Husband,Oscar Wilde,"Classics, Plays, Fiction, Drama, Humor, 19th C...",0.187767
6,The Cat in the Hat and Other Dr. Seuss Favorites,"Dr. Seuss, Various","Childrens, Picture Books, Classics, Fiction, P...",0.179077
1,The House at Pooh Corner,"A.A. Milne, Ernest H. Shepard","Classics, Childrens, Fiction, Fantasy, Animals...",0.176937


# Weighted Hybrid

In [142]:
def weighted_hybrid_recommendations(query, df, type="combined", n_recommendations=5, content_weight=0.5, cf_weight=0.5):
    # Adjust the number of recommendations fetched from each model to account for overlaps
    n_content = int(n_recommendations * (content_weight / (content_weight + cf_weight)))
    n_cf = n_recommendations - n_content
    
    # Select the appropriate DataFrame based on df_identifier
    if 'BookID' in df.columns:
        df_content = gbx_content
        df_cf = gbx_cf
    else:
        df_content = bx_content
        df_cf = bx_cf

    if type.lower() == "title":
        cont_type = "title"
        cf_type = "title"
    elif type.lower() == "author":
        cont_type = "author"
        cf_type = "title"
    elif type.lower() == "genres":
        cont_type = "genres"
        cf_type = "genres"
    elif type.lower() == "combined":
        cont_type = "combined"
        cf_type = "title"
    
    # Get recommendations from content-based model
    content_recs = content_based_recommendations(query, df_content, type = cont_type, n_recommendations=n_content)
    
    # Get recommendations from collaborative filtering model
    cf_recs = collaborative_recommendations(query, df_cf, type = cf_type, n_recommendations=n_cf)
    # cf_recs = collaborative_recommendations(book_title, df_cf, type="genres", n_recommendations=n_cf)
    
    # Combine recommendations
    combined_recs = pd.concat([content_recs, cf_recs]).drop_duplicates(subset=['Title', 'Author'])
    # print(combined_recs)
    

    combined_recommendations = cosine_sim(combined_recs, book_title=query)  # Calculate cosine similarity
    sorted_recommendations = combined_recommendations.sort_values(by='Cosine Similarity', ascending=False)
    top_recommendations = sorted_recommendations.head(n_recommendations).reset_index(drop=True)
    final_recs = top_recommendations.loc[:, ['Title', 'Author', 'Genres']]
    return final_recs




    # # If there are more recommendations than needed, prioritize content-based recommendations
    # if len(combined_recs) > n_recommendations:
    #     combined_recs = combined_recs.head(n_recommendations)
    
    # return combined_recs.reset_index(drop=True)


In [83]:
book_title = "Fifty Shades of Grey"
weighted_hybrid_recommendations(book_title, df = gbx_hybrid, n_recommendations=5)

Your query:-
Query: Fifty Shades of Grey
Genres: Fiction, Contemporary, British Literature, Novels, LGBT, Coming Of Age, 20th Century


,Title,Author,Genres,Cosine Similarity
8800,Fifty Shades Freed,E.L. James,"Romance, Erotica, Fiction, BDSM, Adult, Contem...",1.000000
9100,Fifty Shades Darker,E.L. James,"Romance, Fiction, Erotica, BDSM, Adult, Contem...",1.000000
0,What I Loved,Siri Hustvedt,"Fiction, Contemporary, Art, Literary Fiction, ...",0.166593
3800,"Eat, pray, love: one woman's search for everyt...",Elizabeth Gilbert,"Nonfiction, Memoir, Travel, Biography, Romance...",0.142316
1,The Buddha of Suburbia,Hanif Kureishi,"Fiction, Contemporary, British Literature, Nov...",0.137919


In [117]:
weighted_hybrid_recommendations("Fifty Shades Of Grey", df=gbx_hybrid, type="Genres", n_recommendations=5)

Your query:-
Query: Fifty Shades Of Grey


,Title,Author,Genres,Cosine Similarity
0,Fifty Shades Freed,E.L. James,"Romance, Erotica, Fiction, BDSM, Adult, Contem...",1.000000
1,Fifty Shades Darker,E.L. James,"Romance, Fiction, Erotica, BDSM, Adult, Contem...",1.000000
2,"Eat, pray, love: one woman's search for everyt...",Elizabeth Gilbert,"Nonfiction, Memoir, Travel, Biography, Romance...",0.130593
3,A Hat Full of Sky,Terry Pratchett,"Fantasy, Fiction, Young Adult, Humor, Children...",0.103518
4,Words of Radiance,Brandon Sanderson,"Fantasy, Fiction, Epic Fantasy, High Fantasy, ...",0.079192


In [119]:
weighted_hybrid_recommendations("Fifty Shades Of Grey", df=gbx_hybrid, type="Genres", n_recommendations=5)

Your query:-
Query: Fifty Shades Of Grey


,Title,Author,Genres,Cosine Similarity
0,Fifty Shades Freed,E.L. James,"Romance, Erotica, Fiction, BDSM, Adult, Contem...",1.000000
1,Fifty Shades Darker,E.L. James,"Romance, Fiction, Erotica, BDSM, Adult, Contem...",1.000000
2,"Eat, pray, love: one woman's search for everyt...",Elizabeth Gilbert,"Nonfiction, Memoir, Travel, Biography, Romance...",0.130593
3,A Hat Full of Sky,Terry Pratchett,"Fantasy, Fiction, Young Adult, Humor, Children...",0.103518
4,Words of Radiance,Brandon Sanderson,"Fantasy, Fiction, Epic Fantasy, High Fantasy, ...",0.079192


In [122]:
weighted_hybrid_recommendations("Fifty Shades Of Grey", df=gbx_hybrid, type="combined", n_recommendations=5)

Your query:-
Query: Fifty Shades Of Grey


,Title,Author,Genres,Cosine Similarity
0,Fifty Shades Freed,E.L. James,"Romance, Erotica, Fiction, BDSM, Adult, Contem...",1.000000
1,Fifty Shades Darker,E.L. James,"Romance, Fiction, Erotica, BDSM, Adult, Contem...",1.000000
2,What I Loved,Siri Hustvedt,"Fiction, Contemporary, Art, Literary Fiction, ...",0.166593
3,"Eat, pray, love: one woman's search for everyt...",Elizabeth Gilbert,"Nonfiction, Memoir, Travel, Biography, Romance...",0.142316
4,The Buddha of Suburbia,Hanif Kureishi,"Fiction, Contemporary, British Literature, Nov...",0.137919


In [81]:
book_title = "Fifty Shades of Grey"
weighted_hybrid_recommendations(book_title, df = gbx_hybrid, n_recommendations=5, content_weight=0.3, cf_weight=0.7)

,Title,Author,Genres
0,What I Loved,Siri Hustvedt,"Fiction, Contemporary, Art, Literary Fiction, ..."
1,90 Minutes in Heaven,"Don Piper, Cecil Murphey","Christian, Nonfiction, Religion, Spirituality,..."
2,Les Fleurs du mal,"Charles Baudelaire, Richard Howard","Classics, France, Poetry, French Literature, F..."
3,Ben-Hur: A Tale of the Christ,Lew Wallace,"Classics, Historical Fiction, Fiction, Christi..."
4,"Darkness, Take My Hand",Dennis Lehane,"Mystery, Fiction, Crime, Thriller, Mystery Thr..."


In [84]:
book_title = "Fifty Shades of Grey"
weighted_hybrid_recommendations(book_title, df = gbx_hybrid, n_recommendations=5, content_weight=0.3, cf_weight=0.7)

,Title,Author,Genres
0,What I Loved,Siri Hustvedt,"Fiction, Contemporary, Art, Literary Fiction, ..."
1,Fifty Shades Freed,E.L. James,"Romance, Erotica, Fiction, BDSM, Adult, Contem..."
2,Fifty Shades Darker,E.L. James,"Romance, Fiction, Erotica, BDSM, Adult, Contem..."
3,Deception Point,Dan Brown,"Fiction, Thriller, Mystery, Suspense, Mystery ..."
4,Digital Fortress,Dan Brown,"Fiction, Thriller, Mystery, Suspense, Mystery ..."


In [174]:
# sparsity = 1.0-len(gbx_cf['User_Rating'])/float(gbx_cf['User_Rating'].shape[0]*8195)
# print ('The sparsity level of Goodbooks-10k dataset is ' +  str(sparsity*100) + ' %')

The sparsity level of Goodbooks-10k dataset is 99.98779743746186 %


In [177]:
# sparsity_bx = 1.0-len(bx_cf['User_Rating'])/float(bx_cf['User_Rating'].shape[0]*18467)
# print ('The sparsity level of Book Crossing dataset is ' +  str(sparsity_bx*100) + ' %')

The sparsity level of Book Crossing dataset is 99.99458493528998 %


In [ ]:
# def switching_hybrid_recommendations(book_title, df, n_recommendations):
    

In [37]:
# def popularity_based_recommendations(df, column_type, num_recommendations=5):

#     if column_type not in df.columns:
#         raise ValueError(f"Column type '{column_type}' not found in DataFrame.")
    
#     # Assuming there's a 'Rating' column to calculate average rating for popularity.
#     # If there's no such column, you could count the number of times each item appears as a proxy for popularity.
#     if 'Average_Rating' in df.columns:
#         # Group by the specified column and calculate the average rating.
#         popularity = df.groupby(column_type)['User_Rating'].mean()
#     else:
#         # Count the occurrences of each item in the specified column.
#         popularity = df[column_type].value_counts()

#     # book_popularity = df.groupby('cleaned_title')['User_Rating'].count().sort_values(ascending=False).head(n)
    
#     # Sort the items based on popularity and get the top N items.
#     top_items = popularity.nlargest(num_recommendations).index.tolist()
    
#     # Fetch the details of the top N items.
#     # Assuming each item in the specified column is unique in the DataFrame,
#     # or you might want to drop duplicates based on the 'column_type' if multiple entries exist.
#     recommendations = df[df[column_type].isin(top_items)]
    
#     # If you want to ensure the order of recommendations matches the order of top_items,
#     # this step can be necessary as 'isin' does not guarantee order.
#     # recommendations = recommendations.set_index(column_type).reindex(top_items).reset_index()
    
#     # Depending on the DataFrame structure, you might need to drop duplicates.
#     recommendations = recommendations.drop_duplicates(subset='cleaned_title')
#     selected_cols = ['Title', 'Author', 'Genres', 'Average_Rating'] 
#     final_recommendations = df[selected_cols].copy()
#     final_recommendations.drop_duplicates(subset='Title')
    
#     return final_recommendations.head(num_recommendations)


In [125]:
# popularity_based_recommendations(bx_hybrid, "Title", num_recommendations=5)

In [126]:
# popularity_based_recommendations(bx_hybrid, "Title", num_recommendations=5)

In [127]:
# popularity_based_recommendations(gbx_hybrid, "Title", num_recommendations=5)

# HYBRID MODEL WITH BERT, TFIDF, AND CF FEATURES

In [ ]:
# from sklearn.preprocessing import StandardScaler

In [ ]:
# def compute_combined_features(df, column_type='new_combined'):
#     # Computing TF-IDF Features
#     tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
#     tfidf_features = tfidf_vectorizer.fit_transform(df[column_type])

#     # Concatenating BERT and TF-IDF Features
#     bert_features = np.stack(df['combined_embeddings'].to_numpy())  # Use stack for an array of arrays
#     combined_features = hstack([bert_features, tfidf_features])

#     return combined_features, tfidf_vectorizer

In [ ]:
# def get_cf_features(user_item_matrix):
#     # Transpose the user-item matrix to align with the books dimension
#     cf_features = user_item_matrix.T.values
#     return cf_features

In [ ]:
# def combined_hybrid_recommendations(book_title, df, top_k=5):

#     cleaned_query = clean_text(book_title)

#     if 'BookID' in df.columns:
#         user_item_matrix = user_item_matrix_gbx
#     else:
#         user_item_matrix = user_item_matrix_bx
#     # Compute content-based features
#     content_features, tfidf_vectorizer = compute_combined_features(df, 'new_combined')
    
#     # Compute CF features
#     cf_features = get_cf_features(user_item_matrix)
    
#     # Standardize and combine features
#     scaler = StandardScaler()
#     combined_features = np.hstack((content_features.toarray(), cf_features)) if not isinstance(content_features, np.ndarray) else np.hstack((content_features, cf_features))
#     standardized_features = scaler.fit_transform(combined_features)
    
#     # Fit k-NN on combined standardized features
#     knn_model = NearestNeighbors(n_neighbors=top_k, metric='cosine')
#     knn_model.fit(standardized_features)
    
#     # Transform query to combined feature space
#     query_index = df.index[df['cleaned_title']== cleaned_query].tolist()
#     if query_index:
#         query_index = query_index[0]
#         query_feature = standardized_features[query_index].reshape(1, -1)
        
#         # Find nearest neighbors
#         distances, indices = knn_model.kneighbors(query_feature, n_neighbors=top_k+1)
        
#         # Exclude the query book itself from recommendations
#         recommended_indices = indices.flatten()
#         recommended_indices = recommended_indices[recommended_indices != query_index][:top_k]  # Adjust to exclude self and limit to top_k
        
#         # Prepare the recommendation frame
#         recommended_books = df.iloc[recommended_indices][['Title', 'Author', 'Genres']]
        
#         return recommended_books.reset_index(drop=True)
#     else:
#         return "Book title not found in the dataset."

In [45]:
item_collaborative_recommendations(book_title="looking for alaska", df=gbx_cf, n_recommendations=5)

,Title,Author,Genres
500,The Fault in Our Stars,John Green,"Young Adult, Fiction, Contemporary, Realistic ..."
1100,Divergent,Veronica Roth,"Young Adult, Dystopia, Fantasy, Fiction, Scien..."
6300,The Perks of Being a Wallflower,Stephen Chbosky,"Young Adult, Fiction, Coming Of Age, Classics,..."
8000,Paper Towns,John Green,"Young Adult, Fiction, Contemporary, Romance, M..."
8300,The Maze Runner,James Dashner,"Young Adult, Dystopia, Science Fiction, Fictio..."


In [51]:
weighted_hybrid_recommendations(query="the problem with forever", df=gbx_hybrid, type="author", n_recommendations=10)

Your query:-
Query: the problem with forever
Genres: Nonfiction, Spirituality, Buddhism, Philosophy, Self Help, Religion, Psychology


,Title,Author,Genres
0,The Single Girl's To-Do List,Lindsey Kelk,"Chick Lit, Romance, Contemporary, Fiction, Adu..."
1,Bee Season,Myla Goldberg,"Fiction, Jewish, Contemporary, Adult Fiction, ..."
2,"The Field of Swords (Emperor, #3)",Conn Iggulden,"Historical Fiction, Fiction, Historical, War, ..."
3,Brilliance,Marcus Sakey,"Thriller, Fiction, Mystery, Action, Espionage,..."
4,The Slap,Christos Tsiolkas,"Fiction, Australia, Contemporary, Book Club, N..."
5,The Shop on Blossom Street,Debbie Macomber,"Romance, Fiction, Chick Lit, Knitting, Contemp..."
6,Peace Is Every Step: The Path of Mindfulness i...,"Thich Nhat Hanh, Arnold Kotler, Dalai Lama XIV","Nonfiction, Spirituality, Buddhism, Philosophy..."
7,Broken prince,Erin Watt,"Romance, New Adult, Contemporary, Young Adult,..."
8,After We Collided,Anna Todd,"Romance, New Adult, Young Adult, Contemporary,..."
9,Don't Look Back,Jennifer L. Armentrout,"Young Adult, Mystery, Contemporary, Romance, T..."


# DISPLAYING RECOMMENDATIONS

In [52]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [80]:
def display_book_recommendations():
    # Text field input for the query
    query_input = widgets.Text(
        value='',
        placeholder='Enter query',
        description='Name:',
        disabled=False
    )
    
    # Dropdown to select the recommendation type
    rec_model = widgets.Dropdown(
        options=['Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighted Hybrid'],
        value='Select',
        description='Model:',
    )

    data_frame = widgets.Dropdown(
        options = ['Select', 'Book-Crossing', 'Goodbooks-10k'],
        value = 'Select',
        description = 'Dataset'
    )

    type_rec = widgets.Dropdown(
        options = ['Select', 'Title-based', 'Author-based', 'Genre-based', 'Combined'],
        value = 'Select',
        description = 'Recommendation Type'
    )

    num_recs = widgets.Dropdown(
        options = ['Select', '1', '2','3','4','5','6','7','8','9','10'],
        value = 'Select',
        description = 'Number'
    )

    
    # Button to generate recommendations
    generate_button = widgets.Button(description="Generate Recommendations")
    
    # Output widget to display the recommendations
    output = widgets.Output()
    
    def on_generate_button_clicked(b):
        with output:
            clear_output()
            query = query_input.value
            model = rec_model.value
            dataframe = data_frame.value
            rec_type = type_rec.value
            n_recommendations = int(num_recs.value)  # You can modify this to be user-selectable as well
            # similarity_threshold = 0.02  # Default threshold, adjust as necessary

            # recommendations, message = pd.DataFrame(), ""
            recommendations = pd.DataFrame()
            

            if query:  # Check if query is not empty

                if dataframe == "Book-Crossing":
                    df = bx_hybrid
                elif dataframe == "Goodbooks-10k":
                    df = gbx_hybrid
                
                if rec_type == "Title-based":
                    cont_type = "title"
                    cf_type = "title"
                elif rec_type == "Author-based":
                    cont_type = "author"
                    cf_type = "title"
                elif rec_type == "Genre-based":
                    cont_type = "genres"
                    cf_type = "genres"
                elif rec_type == "Combined":
                    cont_type = "combined"
                    cf_type = "title"


                if model == 'Content-Based':
                    recommendations = content_based_recommendations(query, df, cont_type, n_recommendations)
                elif model == 'Collaborative':  # Collaborative
                    recommendations= collaborative_recommendations(query, df, cf_type, n_recommendations)
                elif model == 'Cascade Hybrid': 
                    recommendations = cascade_hybrid_recss(query, df, cont_type, n_recommendations)
                elif model == 'Weighted Hybrid': 
                    recommendations= weighted_hybrid_recommendations(query, df, cont_type, n_recommendations)
                else:
                    print("Please select a valid recommendation model.")
                
                if recommendations.empty:
                    print("No recommendations found")
                else:
                    display(recommendations)

    
    generate_button.on_click(on_generate_button_clicked)
    
    # Display the widgets
    display(query_input, rec_model, data_frame, type_rec, num_recs, generate_button, output)

In [105]:
display_book_recommendations()

Text(value='', description='Name:', placeholder='Enter query')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighte…

Dropdown(description='Dataset', options=('Select', 'Book-Crossing', 'Goodbooks-10k'), value='Select')

Dropdown(description='Recommendation Type', options=('Select', 'Title-based', 'Author-based', 'Genre-based', '…

Dropdown(description='Number', options=('Select', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'), value='S…

Button(description='Generate Recommendations', style=ButtonStyle())

Output()

In [107]:
display_book_recommendations()

Text(value='', description='Name:', placeholder='Enter query')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighte…

Dropdown(description='Dataset', options=('Select', 'Book-Crossing', 'Goodbooks-10k'), value='Select')

Dropdown(description='Recommendation Type', options=('Select', 'Title-based', 'Author-based', 'Genre-based', '…

Dropdown(description='Number', options=('Select', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'), value='S…

Button(description='Generate Recommendations', style=ButtonStyle())

Output()

In [111]:
display_book_recommendations()

Text(value='', description='Name:', placeholder='Enter query')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighte…

Dropdown(description='Dataset', options=('Select', 'Book-Crossing', 'Goodbooks-10k'), value='Select')

Dropdown(description='Recommendation Type', options=('Select', 'Title-based', 'Author-based', 'Genre-based', '…

Dropdown(description='Number', options=('Select', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'), value='S…

Button(description='Generate Recommendations', style=ButtonStyle())

Output()

In [113]:
display_book_recommendations()

Text(value='', description='Name:', placeholder='Enter query')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighte…

Dropdown(description='Dataset', options=('Select', 'Book-Crossing', 'Goodbooks-10k'), value='Select')

Dropdown(description='Recommendation Type', options=('Select', 'Title-based', 'Author-based', 'Genre-based', '…

Dropdown(description='Number', options=('Select', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'), value='S…

Button(description='Generate Recommendations', style=ButtonStyle())

Output()

In [115]:
display_book_recommendations()

Text(value='', description='Name:', placeholder='Enter query')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighte…

Dropdown(description='Dataset', options=('Select', 'Book-Crossing', 'Goodbooks-10k'), value='Select')

Dropdown(description='Recommendation Type', options=('Select', 'Title-based', 'Author-based', 'Genre-based', '…

Dropdown(description='Number', options=('Select', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'), value='S…

Button(description='Generate Recommendations', style=ButtonStyle())

Output()

In [143]:
display_book_recommendations()

Text(value='', description='Name:', placeholder='Enter query')

Dropdown(description='Model:', options=('Select', 'Content-Based', 'Collaborative', 'Cascade Hybrid', 'Weighte…

Dropdown(description='Dataset', options=('Select', 'Book-Crossing', 'Goodbooks-10k'), value='Select')

Dropdown(description='Recommendation Type', options=('Select', 'Title-based', 'Author-based', 'Genre-based', '…

Dropdown(description='Number', options=('Select', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'), value='S…

Button(description='Generate Recommendations', style=ButtonStyle())

Output()